In [61]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [47]:
dataset_path = "dataset_amyloid.pickle"
with open(dataset_path, 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

In [48]:
# print (X_train.shape)
# print (X_test.shape)
# print(X_train[0])
# print(X_train[0:])

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# print (X_train.shape)
# print (X_test.shape)
# print(X_train[0])
# print(X_train[0:])

In [55]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Dense(32, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      layers.Dense(16, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(140, activation="sigmoid")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

In [49]:
class Autoencoder1(Model):
  def __init__(self, latent_dim, shape):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.shape = shape
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(tf.math.reduce_prod(shape).numpy(), activation='sigmoid'),
      layers.Reshape(shape)
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


shape = X_test.shape[1:]
latent_dim = 64
autoencoder = Autoencoder1(latent_dim, shape)

In [62]:
# Defining Autoencoder model
class Autoencoder2(nn.Module):
   def __init__(self, input_size, encoding_dim):
       super(Autoencoder, self).__init__()
       self.encoder = nn.Sequential(
           nn.Linear(input_size, 16),
           nn.ReLU(),
           nn.Linear(16, encoding_dim),
           nn.ReLU()
       )
       self.decoder = nn.Sequential(
           nn.Linear(encoding_dim, 16),
           nn.ReLU(),
           nn.Linear(16, input_size),
           nn.Sigmoid()
       )

   def forward(self, x):
       x = self.encoder(x)
       x = self.decoder(x)
       return x

In [56]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [63]:
# Setting random seed for reproducibility
torch.manual_seed(42)

input_size = X.shape[1]  # Number of input features
encoding_dim = 3  # Desired number of output dimensions
model = Autoencoder(input_size, encoding_dim)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

# Training the autoencoder
num_epochs = 20
for epoch in range(num_epochs):
   # Forward pass
   outputs = model(X_tensor)
   loss = criterion(outputs, X_tensor)

   # Backward pass and optimization
   optimizer.zero_grad()
   loss.backward()
   optimizer.step()

   # Loss for each epoch
   print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Encoding the data using the trained autoencoder
encoded_data = model.encoder(X_tensor).detach().numpy()

NameError: name 'X' is not defined

In [57]:
autoencoder.fit(X_train, X_train,
                epochs=10,
                shuffle=True,
                validation_data=(X_test, X_test))

Epoch 1/10


ValueError: Dimensions must be equal, but are 10193 and 140 for '{{node compile_loss/mean_squared_error/sub}} = Sub[T=DT_FLOAT](data_1, anomaly_detector_1/sequential_15_1/dense_19_1/Sigmoid)' with input shapes: [?,10193], [?,140].

In [52]:
encoded_data = autoencoder.encoder(X_test).numpy()
decoded_data = autoencoder.decoder(encoded_imgs).numpy()

In [53]:
print(X_test.shape)
print(X_test[0])
print(X_test[0:])

print(encoded_data.shape)
print(encoded_data[0])
print(encoded_data[0:])

print(decoded_data.shape)
print(decoded_data[0])
print(decoded_data[0:])

(35, 10193)
[0. 0. 0. ... 0. 0. 0.]
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.00392157 0.         ... 0.         0.         0.        ]
 [0.         0.01568628 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(35, 64)
[  0.       102.464584   0.        86.15958   63.324093   0.
   0.        13.621096   0.         0.         0.        95.339806
 120.054344  36.172222 154.66162   25.494608 134.28531    0.
  71.39205    0.        96.422775  37.20052   44.74671   80.33154
 163.20021   84.66713    0.        86.95387   50.4607    82.84745
   0.       172.92311   73.42575  145.74951   66.24916   77.07766
   0.       102.865      0.        76.10905  131.20297  131.87607
   0.       120.00564   48.44669    0.         0.         